In [9]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MODEL_NAME = "Qwen/Qwen3-1.8B-Instruct"  # use 4B if GPU is strong
DATA_PATH = "/content/drive/MyDrive/NLP/news.csv"
OUTPUT_DIR = "/content/qwen3_burmese_headline"

In [7]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Headline Generator Dataset/headline_corpus.csv")
df.head(5)

X = df["text"]
y = df['headline']

In [10]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 2) Split temp into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# Check sizes
print("Train:", len(X_train))
print("Validation:", len(X_val))
print("Test:", len(X_test))

Train: 11853
Validation: 2540
Test: 2541


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
tokenized_ds = dataset.map(tokenize, batched=True)

In [ ]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=4,
    fp16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_ds["train"],
    tokenizer=tokenizer,
    args=args
)

trainer.train()


In [ ]:
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)